In [1]:
!pip install import-ipynb   # or pip install --quiet import-ipynb


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install hf_xet torch torchvision transformers tqdm requests datasets accelerate bitsandbytes tensorboard torch-tb-profiler openai anthropic google-generativeai


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import import_ipynb                     # enables ipynb imports
from enn_helpers import LlamaBaseNet, EpiNet, PriorNet, EpistemicNN
import os
import json
import torch
import platform
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer, logging

from torch.utils.tensorboard import SummaryWriter


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Using device: mps
Torch: 2.7.0   Free RAM: 3.832922112 GB
MPS cap: True
macOS-13.4-arm64-arm-64bit 2.7.0
MPS available: True
Loading HaluEval dataset from Hugging Face...
Loading qa dataset...
Loading dialogue dataset...
Loading summarization dataset...
Loading general dataset...
Merging all training data...
Merging all test data...
HaluEval dataset preparation complete!
Train data: data/halueval/train.jsonl
Test data: data/halueval/test.jsonl


In [ ]:
# Hyperparameters
num_classes = 2  # Binary classification for hallucination detection
z_dim = 16
hidden_dims = [128, 64]
lr = 1e-5  # Lower learning rate for LLM fine-tuning
epochs = 3  # Reduce epochs for faster training with large model
λ = 1e-5
batch_size = 1  # Smaller batch size for large model
max_length = 128
import random

# Choose a Llama model - use a smaller version if memory is limited
model_name = "meta-llama/Llama-2-7b-hf"

class HaluEvalDataset(Dataset):
    def __init__(self, data_items, tokenizer):
        self.data = data_items
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        # ---------- 1. pick the *content* string ----------
        if "question" in item:          # QA
            content = f"Context: {item['knowledge']}\nQuestion: {item['question']}"
            gold   = item["right_answer"]
            halluc = item["hallucinated_answer"]

        elif "dialogue_history" in item:      # Dialogue
            content = f"Context: {item['knowledge']}\nDialogue: {item['dialogue_history']}"
            gold   = item["right_response"]
            halluc = item["hallucinated_response"]

        elif "document" in item:              # Summarisation
            content = f"Document: {item['document']}"
            gold   = item["right_summary"]
            halluc = item["hallucinated_summary"]

        else:                                 # General split
            content = f"User query: {item['user_query']}"
            # general‐split has only one answer plus a Yes/No label
            gold   = item["chatgpt_response"]
            halluc = None                     # there is no alt-answer

        # ---------- 2. choose gold vs hallucinated version ----------
        if halluc is not None and torch.rand(1).item() > 0.5:
            text  = f"{content}\nAnswer: {halluc}"
            label = 1            # hallucination
        else:
            text  = f"{content}\nAnswer: {gold}"
            label = 0            # factual

        # ---------- 3. tokenise ----------
        enc = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=max_length,
            return_tensors="pt",
        )

        return enc["input_ids"].squeeze(0), torch.tensor(label, dtype=torch.long)
    
# Initialize models
print("Initializing models...")
base = LlamaBaseNet(model_name, num_classes).to(device)
feature_dim = base.hidden_size
epinet = EpiNet(feature_dim, z_dim, hidden_dims, num_classes).to(device)
prior = PriorNet(feature_dim, z_dim, num_classes).to(device)
enn = EpistemicNN(base, epinet, prior).to(device)

# Create train/test splits from the loaded HaluEval dataset
print("Loading dataset...")
all_data = []
categories = ["qa", "dialogue", "summarization", "general"]

for category in categories:
    print(f"Loading {category} dataset...")
    dataset = load_dataset("pminervini/HaluEval", category)
    all_data.extend(dataset['data'])

# Shuffle and split the data
random.seed(42)
random.shuffle(all_data)
split_idx = int(0.8 * len(all_data))
train_data = all_data[:split_idx]
test_data = all_data[split_idx:]

# -------------------- build proper Dataset objects --------------------
train_dataset = HaluEvalDataset(train_data, base.tokenizer)
test_dataset  = HaluEvalDataset(test_data,  base.tokenizer)

trainloader = DataLoader(train_dataset,
                         batch_size=batch_size,
                         shuffle=True)

testloader  = DataLoader(test_dataset,
                         batch_size=batch_size,
                         shuffle=False)

print(f"Training samples: {len(train_data)}")
print(f"Testing samples: {len(test_data)}")

In [ ]:
import torch, gc

torch.mps.empty_cache()   # releases unused cached blocks
gc.collect()              # Python-side garbage collection


In [ ]:
# Train the model first
print("Training model...")
trained_model = train_enn(enn, trainloader, epochs, lr, λ, z_dim, device, tb_writer)

# Then evaluate the model
print("Evaluating model...")
accuracy, uncertainty = evaluate_enn(trained_model, testloader, z_dim, device)

# Save the model
torch.save(trained_model.state_dict(), "trained_enn_halueval_model.pt")
print("Model saved to trained_enn_halueval_model.pt")
tb_writer.close()